# Unfold a Polygon to create a Translation Surface

In [ ]:
from ipyvue_flatsurf import FlatSurface
from flatsurf import translation_surfaces, polygons, similarity_surfaces

In [ ]:
t = polygons.triangle(1, 1, 3)
B = similarity_surfaces.billiard(t)
S = B.minimal_cover('translation')

# A Visulation for a Flow Decomposition

In [ ]:
from flatsurf import GL2ROrbitClosure
O = GL2ROrbitClosure(S)
D = O.decomposition((0, 1))

In [ ]:
V = FlatSurface(D)
V

## Use the Same Glueings for Another Flow Decomposition

In [ ]:
import ipywidgets

D = O.decomposition((1, 0))
W = FlatSurface(D)
ipywidgets.link((V,'inner'), (W, 'forced'))
W